In [52]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.linalg import cho_solve, cho_factor

## Useful functions from pose_graph_jacobian.ipynb
Just functions for `Exp` and `Log` maps, `cross` product matrix function and calculation of the jacobian

In [53]:
def np_cross(x):
    return np.array([
        [0,-x[2],x[1]],
        [x[2],0,-x[0]],
        [-x[1],x[0],0]
    ])

def np_exp(x):
    theta = np.linalg.norm(x[3:])
    if theta == 0:
        R = np.eye(3)
        t = x[:3]
    else:
        a = np.sin(theta)/theta
        b = (1 - np.cos(theta))/(theta*theta)
        c = (theta - np.sin(theta))/(theta**3)
        w = np_cross(x[3:])
        R = np.eye(3) + a*w + b*w@w

        t = (np.eye(3) + b*w + c*w@w)@x[:3]
    
    mat = np.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def np_log(T):
    theta = np.arccos((np.trace(T) - 2) / 2)
    # theta = np.arccos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return T[0, 3], T[1, 3], T[2, 3], 0, 0, 0

    a = np.sin(theta)/theta
    b = (1 - np.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = np_cross(np.array([x4,x5,x6]))
    u = (np.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return u[0], u[1], u[2], x4, x5, x6

In [54]:
def np_adjoint(X):
    """X is a 4x4 matrix not a 6 Vector"""
    mat = np.zeros((6,6))
    mat[:3,:3] = X[:3,:3]
    mat[3:,3:] = X[:3,:3]
    mat[:3,3:] = np_cross(X[:3,3])@X[:3,:3]

    return mat.T

def np_error(Z, Xi, Xj):
    return np_log(np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj))

def np_exp_error(Z, Xi, Xj):
    return np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj)

In [55]:
def box_plus(a, b):
    return np_log(np_exp(a)@np_exp(b))

def box_minus(a, b):
    return np_log(np.linalg.inv(np_exp(b))@np_exp(a))

In [56]:
def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = np_exp(Xi), np_exp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@np_cross(eXj[:3,3])

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = np_cross(eXi[:3, 3])@eXi[:3,:3]

    return -(A@B).T

def Je_analytic_Xj(Z, Xi, Xj):
    return np.eye(6)

## Setting up the pose graph problem
The plan here is to define a pose graph problem that is simple to solve and has a known correct solution - i.e. I can check that I'm converging on the right spot. To start with I think I'm going to go with 3 fully connected poses. I then calculate the correct measurements between them as my measurements, and set the information matrices to something reasonable. After that, displace the points and use these new points as my starting off point for the optimisation - if it correctly comes back to the original position then I'll know I've done something right and can move onto progressively harder problems.

In [57]:
poses = [(np.random.random(6) - 0.5)*2 for _ in range(3)]
measurements = {(i,j):np_log(np.linalg.inv(np_exp(poses[i]))@np_exp(poses[j])) for (i, j) in combinations(range(3), 2)}

eps = 0.05
disturbed_poses = [p + (np.random.normal(0, eps, 6) - 0.5) for p in poses]

infos = {k:np.linalg.inv(np.eye(6)*0.05) for k in measurements.keys()}

Now that we have our poses, measurements and information matrices its time to get on with writing the actual optimisation code. We're going to use Gauss-Newton I think so the update rule is going to look like this:

$$\Delta x = (J^TJ)^{-1}J^Tr$$

In [58]:
def make_parts(poses, measurements, infos):
    H = np.zeros((6*len(poses), 6*len(poses)))
    b = np.zeros(6*len(poses))

    H[:6, :6] += np.eye(6)

    for (i, j), Zij in measurements.items():
        Xi, Xj = poses[i], poses[j]
        omega = infos[(i, j)]

        e = np_error(Zij, Xi, Xj)
        A = Je_analytic_Xi(Zij, Xi, Xj)
        B = Je_analytic_Xj(Zij, Xi, Xj)

        H[i:i+6, i:i+6] += A.T@omega@A
        H[i:i+6, j:j+6] += A.T@omega@B
        H[j:j+6, i:i+6] += B.T@omega@A
        H[j:j+6, j:j+6] += B.T@omega@B

        b[i:i+6] += A.T@omega@e
        b[j:j+6] += B.T@omega@e

    return H, b

H, b = make_parts(disturbed_poses, measurements, infos)

H.shape, b.shape


((18, 18), (18,))

In [59]:
new_poses = disturbed_poses[:]

for i in range(1):
    H, b = make_parts(new_poses, measurements, infos)
    delta = cho_solve(cho_factor(H), b)

    pose_vector = np.concatenate(new_poses)
    for i in range(0, pose_vector.shape[0], 6):
        print(i)
    

LinAlgError: 9-th leading minor of the array is not positive definite